# QED calcutations using SymPy

In [1]:
import sympy as sp
import numpy as np

from sympy import I

from IPython.display import display, Latex
def pprint(S, name=""):
    """Print out a single sympy expression"""
    string = "$$" 
    if name!="": string = string + name + "=" 
    string = string + sp.latex(S) + "$$"
    return display(Latex(string))

To do QED, we need some setup first. First, we need the metric tensor.

In [2]:
dim = 4
g = sp.Matrix.diag([1, -1, -1, -1])
g

Matrix([
[1,  0,  0,  0],
[0, -1,  0,  0],
[0,  0, -1,  0],
[0,  0,  0, -1]])

We also a representation of the gamma matrices. These can either be defined directly from their anticomutating relations,
$$
    \{\gamma^\mu, \gamma^\nu\} = 2 g^{\mu\nu}
$$
or they can be written out explicitly as matrices:
$$
    \gamma^0 = \
    \begin{pmatrix}
        I_2 & 0 \\
        0 & - I_2
    \end{pmatrix}, \quad
        \gamma^i = \
    \begin{pmatrix}
        0 & \sigma^i \\
        -\sigma^i & 0
    \end{pmatrix}.
$$
There is a sympy module for an abstract representation of the  [gamma matrix](https://docs.sympy.org/latest/modules/physics/hep/index.html). This will be often preferable, due as the matrix multiplication can get messy quickly. However, due to some lacking features in the sympy library (mainly the fact that there is no implementation of $\gamma^5$), we will be using an explicit matrix representation. We will follow the conventions of Grifftihs' "Introduction to Elementary Particles". 

In [3]:
# Pauli matrices
s = np.empty((3, 2, 2), dtype=type(sp.Rational(1)))

one = sp.Rational(1)
s[0] = np.array([
    [0, one],
    [one, 0]
])
s[1] = np.array([
    [0, -I],
    [I, 0]
])
s[2] = np.array([
    [one, 0],
    [0, -one]
])

The reason for using numpy arrays to construct the gamma matrices is that the standard sympy.Array type is not mutable, i.e. once constructed, they cannot be change. One could always make new copies, but the NumPy approache is sometimes more convinient.

In [4]:
# Dirac matricies

Oh = np.zeros((2, 2), dtype=type(sp.Rational))
eye = np.identity(2, dtype=type(sp.Rational))

γ = np.empty((dim, dim, dim), dtype=type(sp.Rational(1)))
γ[0] = np.block([
    [eye, Oh],
    [Oh, -eye]
])

for i in range(3):
    γ[i + 1] = np.block([
        [Oh, s[i]],
        [-s[i], Oh]
    ])
    
γ = sp.Array(γ)
γ

[[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]], [[0, 0, 0, 1], [0, 0, 1, 0], [0, -1, 0, 0], [-1, 0, 0, 0]], [[0, 0, 0, -I], [0, 0, I, 0], [0, I, 0, 0], [-I, 0, 0, 0]], [[0, 0, 1, 0], [0, 0, 0, -1], [-1, 0, 0, 0], [0, 1, 0, 0]]]

In [5]:
γ5 = I
for mat in γ:
    γ5 *= sp.Matrix(mat)
γ5

Matrix([
[0, 0, 1, 0],
[0, 0, 0, 1],
[1, 0, 0, 0],
[0, 1, 0, 0]])

Then we need some functions to manipulate matricies, vectors and such.

In [6]:
from sympy import tensorproduct as tp
from sympy import tensorcontraction as tc

def prod(A, v):
    return tc(tp(A, v), (1, 2))

def dot(v1, v2):
    return tc(tp(v1, v2), (0, 1))

def sum(lst):
    """ Must overwrite built in to sum sybols, for some reason """
    s = 0
    for l in lst:
        s += l
    return s

`slash(a)` will return $a^\mu \gamma_\mu = a^\mu g_{\mu\nu}\gamma^\nu $

In [7]:
def slash(a):
    slasha = 0 * γ[0]
    for i in range(dim):
        slasha += γ[i] * g[i, i] * a[i]
    return slasha

The most important feature is the trace function. We could implement a function that only works for expressions of the type $\mathrm{Tr}(\gamma^\mu \gamma^\nu \gamma^\sigma \gamma^\rho)$. However, if we are even more abitious we would like to also handle $\mathrm{Tr}((\gamma^\mu p_ \mu - m) \gamma^\nu)$, where some of the internal space-time indices are contracted, while others are not. To do this, we remember that there are two types of indices, the space time indices $\mu, \nu ...$ but also the internal indices of the gamma matrices. Therefore we can write $\mathrm{Tr}(\gamma^\mu_{ij} \gamma^\nu_{jk}) = \gamma^\mu_{ij} \gamma^\nu_{ji}$, where Einstein summation is used also on the internal $ij$ indices.

The function below takes in a list of matrices, contracted or not, which are to be traced, An example is `(γ, slas(p1), γ, slash(p2))`. It makes a list indicating where the free indices are present, that is uncontracted space-time indices. Then we take the first element of the objects to be traced, and contract the the second internal index with the first of the next obejct. The `i1, i2` objects are to make sure that we skip the free indices. Lastly, we must contract the first and last internal indices, finalizing the trace. This function then returns a SymPy array `A[:, :, ...]` with as many indices as there are free space-time indices. As an example, $\mathrm{Tr}((\gamma^\sigma p_\sigma - m) \gamma^\mu(\gamma^\rho p_\rho - m) \gamma^\nu)$ would return a 2-dimensional array.

In [8]:
def Tr(lst):
    global γ
    
    free_indx = [γ == a for a in lst]
    A = lst[0]
    for i, a in enumerate(lst[1:]):
        A = tp(A, a)
        i1 = sum(free_indx[:i+1])
        i2 = sum(free_indx[:i+2])
        A = tc(A, (1 + i1, i2 + 2))

    last = len(A.shape) - 1
    first = free_indx[0] + 0
    return tc(A, (first, last))

# Mott scattering

We can now recreate calculations in QED. This example is Mott scattering, as Griffiths lays out in example 7.5 to 7.7 

The first Feynmann diagram for electron-muon scattering gives, after using Casimirs trick, the amplitude
$$
\langle | \mathcal{M} |^2 \rangle = 
\frac{g_e^4}{4 (p_1 - p_4)^2} 
\mathrm{Tr}\left[ \gamma^\mu (\gamma_\nu p_3^\nu - m_1) \gamma^\sigma (\gamma_\rho p_1^\rho - m_1) \right] 
\mathrm{Tr}\left[ \gamma_\mu (\gamma_\nu p_2^\nu - m_2) \gamma_\sigma (\gamma_\rho p_4^\rho - m_2) \right]
$$

First, we construc a Array of momenta

In [9]:
num_vec = 4
p = sp.Array([
    sp.symbols("p^{" + str(i) + "}_" + str(j+1)) for j in range(num_vec) 
    for i in range(dim)
]).reshape(num_vec, dim)

p_ = sp.Array([prod(g, p[i]) for i in range(num_vec)])
p

[[p^{0}_1, p^{1}_1, p^{2}_1, p^{3}_1], [p^{0}_2, p^{1}_2, p^{2}_2, p^{3}_2], [p^{0}_3, p^{1}_3, p^{2}_3, p^{3}_3], [p^{0}_4, p^{1}_4, p^{2}_4, p^{3}_4]]

In [10]:
m1, m2 = sp.symbols("m_1, m_2")
Id = sp.Array(sp.eye(4))
slash(p[0]) - m1 * Id

[[-m_1 + p^{0}_1, 0, -p^{3}_1, -p^{1}_1 + I*p^{2}_1], [0, -m_1 + p^{0}_1, -p^{1}_1 - I*p^{2}_1, p^{3}_1], [p^{3}_1, p^{1}_1 - I*p^{2}_1, -m_1 - p^{0}_1, 0], [p^{1}_1 + I*p^{2}_1, -p^{3}_1, 0, -m_1 - p^{0}_1]]

We can then take the traces, and vertify that we indeed get 2 dimensional Array, as we would expect from a trace with 2 free space-time indices.

In [11]:
M1 = Tr((γ, slash(p[0]) - m1 * Id, γ, slash(p[2]) - m1 * Id))
M2 = Tr((γ, slash(p[1]) - m2 * Id, γ, slash(p[3]) - m2 * Id))
sp.simplify(M2)

[[4*m_2**2 + 4*p^{0}_2*p^{0}_4 + 4*p^{1}_2*p^{1}_4 + 4*p^{2}_2*p^{2}_4 + 4*p^{3}_2*p^{3}_4, 4*p^{0}_2*p^{1}_4 + 4*p^{0}_4*p^{1}_2, 4*p^{0}_2*p^{2}_4 + 4*p^{0}_4*p^{2}_2, 4*p^{0}_2*p^{3}_4 + 4*p^{0}_4*p^{3}_2], [4*p^{0}_2*p^{1}_4 + 4*p^{0}_4*p^{1}_2, -4*m_2**2 + 4*p^{0}_2*p^{0}_4 + 4*p^{1}_2*p^{1}_4 - 4*p^{2}_2*p^{2}_4 - 4*p^{3}_2*p^{3}_4, 4*p^{1}_2*p^{2}_4 + 4*p^{1}_4*p^{2}_2, 4*p^{1}_2*p^{3}_4 + 4*p^{1}_4*p^{3}_2], [4*p^{0}_2*p^{2}_4 + 4*p^{0}_4*p^{2}_2, 4*p^{1}_2*p^{2}_4 + 4*p^{1}_4*p^{2}_2, -4*m_2**2 + 4*p^{0}_2*p^{0}_4 - 4*p^{1}_2*p^{1}_4 + 4*p^{2}_2*p^{2}_4 - 4*p^{3}_2*p^{3}_4, 4*p^{2}_2*p^{3}_4 + 4*p^{2}_4*p^{3}_2], [4*p^{0}_2*p^{3}_4 + 4*p^{0}_4*p^{3}_2, 4*p^{1}_2*p^{3}_4 + 4*p^{1}_4*p^{3}_2, 4*p^{2}_2*p^{3}_4 + 4*p^{2}_4*p^{3}_2, -4*m_2**2 + 4*p^{0}_2*p^{0}_4 - 4*p^{1}_2*p^{1}_4 - 4*p^{2}_2*p^{2}_4 + 4*p^{3}_2*p^{3}_4]]

To contract the traces together, we must lower the indices on one of them.

In [12]:
M2_ = prod(g, prod(M2, g))
sp.simplify(M2_)

[[4*m_2**2 + 4*p^{0}_2*p^{0}_4 + 4*p^{1}_2*p^{1}_4 + 4*p^{2}_2*p^{2}_4 + 4*p^{3}_2*p^{3}_4, -4*p^{0}_2*p^{1}_4 - 4*p^{0}_4*p^{1}_2, -4*p^{0}_2*p^{2}_4 - 4*p^{0}_4*p^{2}_2, -4*p^{0}_2*p^{3}_4 - 4*p^{0}_4*p^{3}_2], [-4*p^{0}_2*p^{1}_4 - 4*p^{0}_4*p^{1}_2, -4*m_2**2 + 4*p^{0}_2*p^{0}_4 + 4*p^{1}_2*p^{1}_4 - 4*p^{2}_2*p^{2}_4 - 4*p^{3}_2*p^{3}_4, 4*p^{1}_2*p^{2}_4 + 4*p^{1}_4*p^{2}_2, 4*p^{1}_2*p^{3}_4 + 4*p^{1}_4*p^{3}_2], [-4*p^{0}_2*p^{2}_4 - 4*p^{0}_4*p^{2}_2, 4*p^{1}_2*p^{2}_4 + 4*p^{1}_4*p^{2}_2, -4*m_2**2 + 4*p^{0}_2*p^{0}_4 - 4*p^{1}_2*p^{1}_4 + 4*p^{2}_2*p^{2}_4 - 4*p^{3}_2*p^{3}_4, 4*p^{2}_2*p^{3}_4 + 4*p^{2}_4*p^{3}_2], [-4*p^{0}_2*p^{3}_4 - 4*p^{0}_4*p^{3}_2, 4*p^{1}_2*p^{3}_4 + 4*p^{1}_4*p^{3}_2, 4*p^{2}_2*p^{3}_4 + 4*p^{2}_4*p^{3}_2, -4*m_2**2 + 4*p^{0}_2*p^{0}_4 - 4*p^{1}_2*p^{1}_4 - 4*p^{2}_2*p^{2}_4 + 4*p^{3}_2*p^{3}_4]]

The coefficient is the last thing needed for the amplitude. We must 

In [13]:
ge = sp.symbols("g_e")
coeff = ge**4 /(4 * dot(p[0] - p[2], p_[0] - p_[2])**2)
sp.simplify(coeff)

g_e**4/(4*(-(p^{0}_1 - p^{0}_3)**2 + (p^{1}_1 - p^{1}_3)**2 + (p^{2}_1 - p^{2}_3)**2 + (p^{3}_1 - p^{3}_3)**2)**2)

In [14]:
M = coeff * tc(tc(tp(M1, M2_), (0, 3)), (0, 1))
sp.simplify(M)

g_e**4*(-2*(-(p^{1}_1 - I*p^{2}_1)*(p^{1}_3 - I*p^{2}_3) + (p^{1}_1 + I*p^{2}_1)*(p^{1}_3 + I*p^{2}_3))*(-(p^{1}_2 - I*p^{2}_2)*(p^{1}_4 - I*p^{2}_4) + (p^{1}_2 + I*p^{2}_2)*(p^{1}_4 + I*p^{2}_4)) + 2*(p^{3}_1*(m_1 - p^{0}_3) - p^{3}_1*(m_1 + p^{0}_3) + p^{3}_3*(m_1 - p^{0}_1) - p^{3}_3*(m_1 + p^{0}_1))*(-p^{3}_2*(m_2 - p^{0}_4) + p^{3}_2*(m_2 + p^{0}_4) - p^{3}_4*(m_2 - p^{0}_2) + p^{3}_4*(m_2 + p^{0}_2)) - 2*(-p^{3}_1*(p^{1}_3 - I*p^{2}_3) + p^{3}_1*(p^{1}_3 + I*p^{2}_3) - p^{3}_3*(p^{1}_1 - I*p^{2}_1) + p^{3}_3*(p^{1}_1 + I*p^{2}_1))*(-p^{3}_2*(p^{1}_4 - I*p^{2}_4) + p^{3}_2*(p^{1}_4 + I*p^{2}_4) - p^{3}_4*(p^{1}_2 - I*p^{2}_2) + p^{3}_4*(p^{1}_2 + I*p^{2}_2)) + 2*(p^{3}_1*(p^{1}_3 - I*p^{2}_3) + p^{3}_1*(p^{1}_3 + I*p^{2}_3) + p^{3}_3*(p^{1}_1 - I*p^{2}_1) + p^{3}_3*(p^{1}_1 + I*p^{2}_1))*(p^{3}_2*(p^{1}_4 - I*p^{2}_4) + p^{3}_2*(p^{1}_4 + I*p^{2}_4) + p^{3}_4*(p^{1}_2 - I*p^{2}_2) + p^{3}_4*(p^{1}_2 + I*p^{2}_2)) + (-2*p^{3}_1*p^{3}_3 + (m_1 - p^{0}_1)*(m_1 + p^{0}_3) + (m_1 + p^{

What a horrible mess! This is where it would have been better to have an implementation of the abstract algebraic rules. There is no way to factorize this back into a neat formulat with dot products. However, we may insert give som explicit values for the momenta. As Griffiths shows in Example 7.7, we can set
$$
    p^{(1)} =  [E, p, 0, 0] \\
    p^{(2)} = [M, 0, 0, 0] \\
    p^{(3)} = [E, p \cos(\theta), p \sin(\theta), 0]\\
    p^{(4)} = [M, 0, 0, 0] \\
$$

In [15]:
E, P, theta = sp.symbols("E, p, \\theta")

p_vals = sp.Array([
    [E, P, 0, 0],
    [m2, 0, 0, 0],
    [E, P * sp.cos(theta), P * sp.sin(theta), 0],
    [m2, 0, 0, 0]
])
p_vals

[[E, p, 0, 0], [m_2, 0, 0, 0], [E, p*cos(\theta), p*sin(\theta), 0], [m_2, 0, 0, 0]]

In [16]:
v = p_vals[0] - p_vals[2]
v_ = prod(g, v)
sp.simplify(dot(v, v_)**2)

p**4*(2 - 2*cos(\theta))**2

It is then only a matter of substituting in these values, and using that $E = \sqrt{m_1^2 + p^2}$ to get the final formula.

In [17]:
Ms = M
for i in range(num_vec):
    for j in range(dim):
        Ms = Ms.subs(p[i, j], p_vals[i, j])
    
Mott = sp.simplify(Ms.subs(E, sp.sqrt(m1**2 + P**2)))
Mott

2*g_e**4*m_2**2*(2*m_1**2 + p**2*cos(\theta) + p**2)/(p**4*(cos(\theta) - 1)**2)

With the trig-identities 
$$
    1 - \cos(\theta) = 2 \sin^2(\theta/2), \quad 1 + \cos(\theta) = 2 \cos^2(\theta/2),
$$
we recover eq. (7.130) in Griffiths, save for some factors $c, \hbar$.